In [1]:
import autogen
import pandas as pd
from time import time
from datetime import datetime
import json

flaml.automl is not available. Please install flaml[automl] to enable AutoML functionalities.


In [2]:
config_list_questions = autogen.config_list_from_json("model_config_questions.json")
config_list_questions

[{'model': 'tinyllama:1.1b-chat-v1-q2_K',
  'base_url': 'http://127.0.0.1:11434/v1',
  'api_key': 'ollama',
  'price': [0, 0]}]

In [3]:
config_list_answers = autogen.config_list_from_json("model_config_answers.json")
config_list_answers

[{'model': 'tinyllama:chat',
  'base_url': 'http://127.0.0.1:11434/v1',
  'api_key': 'ollama',
  'price': [0, 0]}]

In [4]:
agent_questions = autogen.AssistantAgent(
    name = "Agent Questions",
    system_message="Your task is to ask a question about computer science.",
    llm_config = {"config_list": config_list_questions},
    code_execution_config = False,
    human_input_mode = "NEVER",
    function_map = None
)
agent_questions

In [5]:
agent_answers = autogen.UserProxyAgent(
    name = "Agent Answers",
    system_message="Your task is to answer a question about computer science and ask for another question.",
    llm_config = {"config_list": config_list_answers},
    code_execution_config = False,
    human_input_mode = "NEVER",
    function_map = None
)
agent_answers

In [6]:
chat_results = agent_questions.initiate_chat(agent_answers, message="What is recursion in computer science?", max_turns=3)

Agent Questions (to Agent Answers):

What is recursion in computer science?

--------------------------------------------------------------------------------
Agent Answers (to Agent Questions):

Recursion is a type of computation that involves invoking itself recursively. In other words, it's a way to apply the same operation or function to smaller and smaller subsets of its input data until some condition is reached that triggers an error or an infinite loop. Recursion is used in many different programming languages and algorithms in computer science to manage a sequence or reduce complexity.

Here's how recursion works when you're coding:

Suppose we have a function named `add_numbers` that takes two numbers as input and adds them together. It can be written like this:

```python
def add_numbers(x, y):
    """Add x and y using recursion"""
    
    if not isinstance(x, int) or not isinstance(y, int) \
    or not (0 <= x < 10 and 0 <= y < 10):
        print("Input values must be integ

---

#### Save Results

In [7]:
timestamp = datetime.fromtimestamp(time()).strftime("%Y-%m-%d_%H-%M-%S")
timestamp

'2024-11-07_12-58-22'

In [8]:
df_chat_history = pd.DataFrame.from_records(chat_results.chat_history)
df_chat_history

,content,role,name
0,What is recursion in computer science?,assistant,Agent Questions
1,Recursion is a type of computation that involv...,user,Agent Answers
2,Variables in Python and other data types are i...,assistant,Agent Questions
3,"Overall, the benefits of using recursion in Py...",user,Agent Answers
4,There are several functions and patterns that ...,assistant,Agent Questions
5,"Yes, there are several patterns and convention...",user,Agent Answers


In [9]:
df_chat_history.to_excel(f"chat_results/chat_history_{timestamp}.xlsx", index=False)

In [10]:
with open(f"chat_results/chat_cost_{timestamp}.json", "w") as json_file:
    json.dump(chat_results.cost, json_file)